In [1]:
from itertools import chain
from glob import iglob
import sqlite3

from h5py import File
import numpy as np
import pandas as pd

In [13]:
def filenames(*runs):
    for r in sorted(set(runs)):
        yield from sorted(iglob("/data/Run_{0:03d}/rawdata/Run_{0:03d}_*.h5".format(r)))


def keys_missing(filename):
    with File(filename, "r") as fp:
        for k in ["/bunches",
                  "/photon_diagnostics/FEL01/I0_monitor/iom_sh_a_pc",
                  "/photon_diagnostics/Spectrometer/hor_spectrum",
                  "/user_laser/delay_line/position",
                  "/user_laser/energy_meter/Energy1",
                  "/digitizer/channel1",
                  "/vmi/andor"]:
            if k not in fp:
                yield {
                    "Filename": filename,
                    "Key missing": k,
                }
                

def tags_range(run):
    fn0, *_, fn1 = filenames(run)
    with File(fn0, "r") as fp:
        tg0 = fp["/bunches"][0]
    with File(fn1, "r") as fp:
        tg1 = fp["/bunches"][-1]
    return {
        "Run": run,
        "Tag first": tg0,
        "Tag last": tg1,
    }

In [12]:
runs = pd.read_excel("../Data/runs.xlsx")
for fn in filenames(*runs[runs["Source"] == "N2"]["Run"]):
    for d in keys_missing(fn):
        with open("../Data/files_bad.txt", "a") as fp:
            fp.write("{Filename},{Key missing}\n".format(**d))

In [18]:
runs = pd.read_excel("../Data/runs.xlsx")
df = pd.DataFrame([tags_range(run) for run in runs[runs["Source"] == "N2"]["Run"]])
df.head()

,Run,Tag first,Tag last
0,197,485824632,485826631
1,198,485899491,485901490
2,199,485995014,485999881
3,200,486010631,486015630
4,201,486050548,486055547
